In [2]:
from __future__ import annotations

import os
import json
import kaggle
import zipfile
import pandas as pd
import tensorflow as tf

2024-01-14 23:47:57.841487: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-14 23:47:57.960268: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 23:47:57.960459: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 23:47:57.960800: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 23:47:58.021601: I tensorflow/core/platform/cpu_feature_g

In [3]:
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [10]:
kaggle.api.authenticate()

In [7]:
!kaggle datasets download -d gpiosenka/100-bird-species

100%|█████████████████████████████████████▉| 1.96G/1.96G [04:23<00:00, 7.28MB/s]
100%|██████████████████████████████████████| 1.96G/1.96G [04:23<00:00, 7.98MB/s]


In [11]:
with zipfile.ZipFile('100-bird-species.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [14]:
os.remove('100-bird-species.zip')
os.remove('EfficientNetB0-525-(224 X 224)- 98.97.h5')

### Change folder name 'PARAKETT  AUKLET' to 'PARAKETT  AKULET' from test, train and valid folder

In [ ]:
os.rename('./test/PARAKETT  AUKLET', './test/PARAKETT  AKULET')
os.rename('./train/PARAKETT  AUKLET', './train/PARAKETT  AKULET')
os.rename('./valid/PARAKETT  AUKLET', './valid/PARAKETT  AKULET')

### Support functions

In [4]:
def load_image_tf(image_path: str, 
                  size: tuple[int, int] = (256, 256)) -> tf.Tensor:
    image = tf.io.read_file(image_path)
    image = tf.io.decode_image(image, expand_animations=False, channels=3)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    
    return tf.image.resize(image, size=[*size])

In [5]:
def zip_images(df: pd.DataFrame,
               size: tuple[int, int] = (256, 256),
               batch_size: int = 32) -> tf.data.Dataset:
    images = tf.data.Dataset.from_tensor_slices(df[df.columns[0]])
    labels = tf.data.Dataset.from_tensor_slices(df[df.columns[1]])
    
    return tf.data.Dataset.zip((images.map(lambda x: load_image_tf(x, size)), labels)).batch(batch_size)

## tf.keras.utils.image_dataset_from_directory


https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

In [6]:
def image_dataset_from_directory(directory: str,
                                 batch_size: int = 32,
                                 image_size: tuple[int, int] = (256, 256),
                                 shuffle: bool = True,
                                 seed: int = None,
                                 validation_split: float = None,
                                 subset: str = None) -> tf.data.Dataset:
    
    # Create files label
    i, class_label, tmp_df = 0, {}, []
    for path, foldres, files in os.walk(directory):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg', '.bmp')):
                class_name = os.path.basename(path)

                if class_name not in class_label:
                        i += 1
                        class_label[class_name] = i
                
                tmp_df.append((f'{path}/{file}', class_label[class_name]))

    df_file_labels = pd.DataFrame(tmp_df, columns=['image_path', 'label'])

    # Shuffle
    if shuffle:
        df_file_labels = df_file_labels.sample(frac=1, random_state=seed)

    if validation_split:
        if not subset:
            raise ValueError('Subset param is not impliment.')
        
        # Sep dataframe by validation_split
        sep = round(df_file_labels.shape[0] * validation_split)

        if subset == 'training':
            return zip_images(df_file_labels[sep:], image_size, batch_size)
        elif subset == 'validation':
            return zip_images(df_file_labels[:sep], image_size, batch_size)
        elif subset == 'both':
            return zip_images(df_file_labels[sep:], image_size, batch_size), zip_images(df_file_labels[:sep], image_size, batch_size)
        
    return zip_images(df_file_labels, image_size, batch_size)


## Use function image_dataset_from_directory

In [7]:
dataset = image_dataset_from_directory('./')

2024-01-14 23:49:22.386770: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-14 23:49:22.391609: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-14 23:49:22.391638: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-14 23:49:22.393361: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-14 23:49:22.393386: I tensorflow/compile

In [12]:
inputs = tf.keras.layers.Input((256, 256, 3), name='input')
x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(inputs)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu")(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(525)(x)

In [13]:
model = tf.keras.Model(inputs=inputs, outputs=x, name='test')
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy', optimizer='adam')

In [14]:
model.summary()

Model: "test"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 125, 125, 32)      9248      
                                                                 
 flatten_1 (Flatten)         (None, 500000)            0         
                                                                 
 dense_1 (Dense)             (None, 525)               262500525 
                                                              

In [15]:
model.fit(dataset, epochs=1)

1023/2809 [=========>....................] - ETA: 1:38 - loss: nan - accuracy: 6.1095e-05

KeyboardInterrupt: 

# Function load_dataset

In [16]:
def load_dataset(path: str, 
                 batch_size: int,
                 image_size: tuple[int, int],
                 shuffle: bool, 
                 split: str) -> tuple[tf.data.Dataset, dict[int, str]]:
    '''
    Given a `path` to a csv index file loads one of the dataset splits. 
    Paths in the index are assumed to be relative to the csv file. 
    The file contains three columns: "filepaths", "labels" and "data set", 
    path to the image, image label and dataset split respectively.

    Arguments:
        path: path to the csv index file
        batch_size: size of batches in the dataset
        image_size: size to resize the images to
        shuffle: whether to shuffle the index. If False original index order is preserved
        split: split to use. One of "train", "valid" or "test"

    Returns:
        The loaded dataset
        A dictionary mapping class indices to class names
    '''

    data = pd.read_csv(path)
    labels = pd.unique(data['labels'])
    labels_to_id = {name: id for id, name in enumerate(labels)}
    
    data.insert(len(data.columns), 'labels_id', [labels_to_id[name] for name in data['labels']], True)

    if shuffle: data = data.sample(frac=1)
    if split: data = data[data['data set'] == split]

    image_dataset = tf.data.Dataset.from_tensor_slices(data['filepaths'])
    label_dataset = tf.data.Dataset.from_tensor_slices(data['labels_id'])
    
    ds = tf.data.Dataset.zip((image_dataset.map(lambda x: load_image_tf(x, image_size)), label_dataset)).batch(batch_size)

    return ds, {id: name for id, name in enumerate(labels)}


In [17]:
dataset, _ = load_dataset('birds.csv', 32, (256, 256), False, None)

In [18]:
model.fit(dataset, epochs=1)

 121/2809 [>.............................] - ETA: 2:26 - loss: nan - accuracy: 0.0421

KeyboardInterrupt: 

# tf.keras.utils.Sequence не успел )))))